In [39]:

import requests
import json
import pandas as pd
# from urllib.parse import quote
import datetime


def get_sm_data(date):

    auth_url = 'https://smartmeter.netz-noe.at/orchestration/Authentication/Login'
    auth_payload = {"user": "SommererPrivatstiftung", "pwd": "SpS*1996"}

    auth_response = requests.post(auth_url, json=auth_payload)
    auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
    auth_xsrf_token = auth_response.cookies['XSRF-Token']
    ### TODO: update date from mobile 
    start_date = '2023-7-10'
    end_date = '2023-9-22'

    data_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Day?meterId=AT0020000000000000000000020826368&day={}&__Host-go4DavidSecurityToken={}".format(date, auth_cookie)

    # data_url = f"https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/BalanceWeek?pointOfConsumption=40565578&startDate=2023-8-22&endDate=2023-8-24"
    # data_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Week?meterId=AT0020000000000000000000020826367&startDate={}&endDate={}".format(start_date, end_date)
    headers = {
        'Cookie': '__Host-go4DavidSecurityToken={}; XSRF-Token={}'.format(auth_cookie, auth_xsrf_token),
    }

    data_response = requests.get(data_url, headers=headers)
    data_response

    data = pd.DataFrame(json.loads(data_response.content.decode('utf-8')))[['meteredValues', 'peakDemandTimes']]
    return data


In [42]:
day1 = get_sm_data('2023-9-18')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
day2 = get_sm_data('2023-9-19')

In [7]:
# Convert all Unicode values in the DataFrame to regular strings
for col in data.columns:
    data[col] = data[col].apply(lambda x: x.encode('utf-8') if isinstance(x, unicode) else x)
    if col == u'peakDemandTimes':
        data[u'peakDemandTimes'] = pd.to_datetime(data[u'peakDemandTimes'])

data.head()

NameError: name 'unicode' is not defined

In [236]:
data.to_csv('smart_meter_data.csv', index=False)

In [109]:
# Define the datetime range
start_date = pd.to_datetime('2023-09-20 00:15:00')
end_date = pd.to_datetime('2023-09-20 01:15:00')

data[(data[u'peakDemandTimes'] <= end_date)][u'meteredValues'].mean()

2.4

In [122]:
datetime.date.today()

datetime.date(2023, 9, 22)

In [238]:
smart_meter_data = pd.read_csv('smart_meter_data.csv')
smart_meter_data[u'peakDemandTimes'] = pd.to_datetime(smart_meter_data[u'peakDemandTimes'])
smart_meter_data[u'peakDemandTimes'].min().date()
smart_meter_data.head()

,meteredValues,peakDemandTimes
0,0.416,2023-09-18 00:15:00
1,0.019,2023-09-18 00:30:00
2,0.000,2023-09-18 00:45:00
3,0.000,2023-09-18 01:00:00
4,0.000,2023-09-18 01:15:00


In [242]:
start_date = '2023-09-18 00:15:00'
end_date = '2023-10-30 00:00:00'


# Define the start_date and end_date
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

# Filter the DataFrame for rows within the desired datetime range
filtered_df = smart_meter_data[(smart_meter_data['peakDemandTimes'] >= start_date) & (smart_meter_data['peakDemandTimes'] <= end_date)]

# Calculate the mean of 'meteredValues' for the filtered rows
filtered_df['meteredValues'].mean()

0.05567708333333334

In [225]:
pd.concat([smart_meter_data, data], ignore_index=True).to_csv('smart_meter_data.csv')


In [222]:
datetime.date.today()

datetime.date(2023, 9, 22)

In [27]:
def smart_meter_data(current_date):
    auth_url = 'https://smartmeter.netz-noe.at/orchestration/Authentication/Login'
    auth_payload = {"user": "SommererPrivatstiftung", "pwd": "SpS*1996"}
    auth_response = requests.post(auth_url, json=auth_payload)
    auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
    auth_xsrf_token = auth_response.cookies['XSRF-Token']
    # current_date = str(datetime.date.today())
    data_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Day?meterId=AT0020000000000000000000020826368&day={}&__Host-go4DavidSecurityToken={}".format(current_date,auth_cookie)
    headers = {
        'Cookie': '__Host-go4DavidSecurityToken={}; XSRF-Token={}'.format(auth_cookie, auth_xsrf_token),
            }

    data_response = requests.get(data_url, headers=headers)
    
    if data_response.status_code == 200:
        print("The status code is 200 (OK).")
    else:
        raise ValueError(f"HTTP request is failing see this code with status code {data_response.status_code}.")
    

    new_data = pd.DataFrame(json.loads(data_response.content))[['meteredValues', 'peakDemandTimes']]

    # Convert all Unicode values in the DataFrame to regular strings
    for col in new_data.columns:
        new_data[col] = new_data[col].apply(lambda x: x.encode('utf-8') if isinstance(x, str) else x)
        if col == u'peakDemandTimes':
            new_data[u'peakDemandTimes'] = pd.to_datetime(new_data[u'peakDemandTimes'])

    smart_meter_data = pd.read_csv('smart_meter_data.csv')
    smart_meter_data[u'peakDemandTimes'] = pd.to_datetime(smart_meter_data[u'peakDemandTimes'])
    if smart_meter_data[u'peakDemandTimes'].max().date() < datetime.date.today():
        concatinated_data = pd.concat([smart_meter_data, new_data], ignore_index=True)

    #saving new data:
    concatinated_data.to_csv('smart_meter_data.csv')
    print("saving new data smartmeter data file to csv file")
    return new_data

In [32]:
def smart_meter_data(current_date):
    auth_url = 'https://smartmeter.netz-noe.at/orchestration/Authentication/Login'
    auth_payload = {"user": "SommererPrivatstiftung", "pwd": "SpS*1996"}
    auth_response = requests.post(auth_url, json=auth_payload)
    auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
    auth_xsrf_token = auth_response.cookies['XSRF-Token']
    # current_date = str(datetime.date.today())
    data_url = "https://smartmeter.netz-noe.at/orchestration/ConsumptionRecord/Day?meterId=AT0020000000000000000000020826368&day={}&__Host-go4DavidSecurityToken={}".format(current_date,auth_cookie)
    headers = {
        'Cookie': '__Host-go4DavidSecurityToken={}; XSRF-Token={}'.format(auth_cookie, auth_xsrf_token),
            }

    data_response = requests.get(data_url, headers=headers)
    
    if data_response.status_code == 200:
        print("The status code is 200 (OK).")
    else:
        raise ValueError(f"HTTP request is failing see this code with status code {data_response.status_code}.")
    

    new_data = pd.DataFrame(json.loads(data_response.content))[['meteredValues', 'peakDemandTimes']]

    # Convert all Unicode values in the DataFrame to regular strings
    for col in new_data.columns:
        if col == u'peakDemandTimes':
            new_data[u'peakDemandTimes'] = pd.to_datetime(new_data[u'peakDemandTimes'])
    return new_data

In [33]:
smart_meter_data('2023-9-18')

ValueError: HTTP request is failing see this code with status code 401.

In [ ]:
import pandas as pd
start_date = '2023-09-18 00:15:00'
end_date = '2023-09-30 00:00:00'
path_to_file = 'smart_meter_data.csv'
# Define the start_date and end_date
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)
smart_meter_data = pd.read_csv(path_to_file)  # Read in the DataFrame



In [7]:
for col in smart_meter_data.columns:
    if col == u'peakDemandTimes':
        smart_meter_data[u'peakDemandTimes'] = pd.to_datetime(smart_meter_data[u'peakDemandTimes'])

In [10]:
# Filter the DataFrame for rows within the desired datetime range
filtered_df = smart_meter_data[(smart_meter_data[u'peakDemandTimes'] >= start_date) & (smart_meter_data[u'peakDemandTimes'] <= end_date)]
# Calculate the mean of 'meteredValues' for the filtered rows
filtered_df['meteredValues'].mean()

0.05567708333333333